<a href="https://colab.research.google.com/github/zhestyatsky/abbyy-nlp-course/blob/main/sem4/hw4_cnn_rnn_for_clf_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 2
Сделайте RNN с другими готовыми эмбеддингами (не обязательно fastText) для решения этой же задачи

In [1]:
%%writefile requirements.txt
pytorch-lightning
scikit-learn
razdel
fasttext
gensim

Writing requirements.txt


In [2]:
!pip install --upgrade -r requirements.txt

     |████████████████████████████████| 675kB 16.2MB/s 
     |████████████████████████████████| 6.8MB 46.2MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 24.2MB 67.3MB/s 
     |████████████████████████████████| 829kB 39.7MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 276kB 55.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3040908 sha256=11b7ee3eebb74fed155911646f11554b485536e3698b3bd8be5b3bd3e53759ff
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=89e04eabab5f9261fbd63b5dc11e6f3e3229944428f532c7914dd03da3a2ed70
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.

## Датасет

In [3]:
!rm negative.csv
!rm positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

rm: cannot remove 'negative.csv': No such file or directory
rm: cannot remove 'positive.csv': No such file or directory
--2020-12-20 21:41:03--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2020-12-20 21:41:03--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce54e90c5a3125dcac2e3609f69.dl.dropboxusercontent.com/cd/0/inline/BFeKfMksfoIqw5G8j6nkVwazPl4cpuhqWQH9es4mi7uM8ksw6JH-Oznt2-Uj3ob7VdRw-519kfkDHRfI5KHAbcIIZWZunRGVpDJcE9lzDRkyhQ/file# [following]
--2020-12-20 21:41:04--  https://uce54e90c5a3125dcac2e3609f69.dl.dropboxusercontent.com/cd/0/inline/BFeKfMksfoIq

In [4]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size], data_negative['text'].values[:sample_size]), axis=0)

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

df_train = pd.DataFrame(columns=['text', 'label'])
df_val = pd.DataFrame(columns=['text', 'label'])
df_test = pd.DataFrame(columns=['text', 'label'])

data = [preprocess_text(t) for t in raw_data]
labels = [1] * sample_size + [0] * sample_size
df_train['text'], df_test['text'], df_train['label'], df_test['label'] = train_test_split(data, labels, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
df_train

,text,label
85913,раньше все встречались у фонтана в гуме а тепе...,1
42792,ни когда не пойму любовь женщины к женщине хот...,1
85556,два сеанса в кино вот что я люблю парам пам па...,1
36360,я нашел мой наряд для кэти перри USER URL via ...,1
154940,USER ууууу всеее развод обидки и все дела,1
...,...,...
73349,закидываю свой вконтакторостер строчками из пе...,1
109259,гребаный понедельник гребанные 4 пары хочу дом...,0
50057,болячки вроде подживают но губы увеличелись в ...,0
5192,USER я знала что она добрая а за что она тебе,1


In [5]:
from razdel import tokenize

train_labels = df_train["label"].tolist()
val_labels = df_val["label"].tolist()
test_labels = df_test["label"].tolist()

texts_set = set(df_train["text"].tolist()) | set(df_val["text"].tolist()) | set(df_test["text"].tolist())

train_texts = [[token.text for token in tokenize(text)] for text in df_train["text"].tolist()]
val_texts = [[token.text for token in tokenize(text)] for text in df_val["text"].tolist()]
test_texts = [[token.text for token in tokenize(text)] for text in df_test["text"].tolist()]

In [6]:
from collections import Counter


class Vocabulary:
    def __init__(self):
        self.word2index = {
            "<pad>": 0,
            "<unk>": 1
        }
        self.index2word = ["<pad>", "<unk>"]

    def build(self, texts, min_count=7):
        words_counter = Counter(token for tokens in texts for token in tokens)
        for word, count in words_counter.most_common():
            if count >= min_count:
                self.word2index[word] = len(self.word2index)
        self.index2word = [word for word, _ in sorted(self.word2index.items(), key=lambda x: x[1])]
    
    def get_pad(self):
        return self.word2index["<pad>"]

    def get_unk(self):
        return self.word2index["<unk>"]
    
    @property
    def size(self):
        return len(self.index2word)
    
    def top(self, n=100):
        return self.index2word[1:n+1]
    
    def get_index(self, word):
        return self.word2index.get(word, self.get_unk())
    
    def get_word(self, index):
        return self.index2word[index]

vocabulary = Vocabulary()
vocabulary.build(train_texts)
assert vocabulary.word2index[vocabulary.index2word[10]] == 10
print(vocabulary.size)
print(vocabulary.top(100))

17198
['<unk>', 'USER', 'не', 'я', 'и', 'в', 'на', 'rt', 'а', 'что', 'URL', 'с', 'как', 'у', 'все', 'меня', 'то', 'это', 'так', 'мне', 'd', 'но', 'ты', 'ну', 'по', 'за', 'еще', 'уже', 'вот', 'да', 'же', 'только', 'сегодня', 'о', 'бы', 'нет', 'когда', 'хочу', 'к', 'очень', 'тебя', 'из', 'он', '3', 'день', 'просто', 'мы', 'будет', '2', 'от', 'было', 'если', 'тебе', 'теперь', 'надо', 'даже', 'тоже', 'завтра', 'кто', 'до', 'там', 'его', '1', 'вообще', 'есть', 'для', 'она', 'сейчас', 'спасибо', 'нас', 'буду', 'почему', 'блин', 'могу', 'люблю', 'без', 'знаю', 'вы', 'они', 'тут', 'или', 'раз', 'мой', 'чем', 'ничего', 'со', 'больше', 'всегда', '5', 'хорошо', 'дома', 'про', 'всем', 'можно', 'ее', 'может', 'год', 'потом', 'был', 'спать']


In [7]:
import os
import torch
import random
import numpy as np

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [8]:
!mkdir russian_conll17
!wget http://vectors.nlpl.eu/repository/20/65.zip
!mv 65.zip russian_conll17/65.zip && cd russian_conll17 && unzip 65.zip

--2020-12-20 21:41:27--  http://vectors.nlpl.eu/repository/20/65.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540031150 (2.4G) [application/zip]
Saving to: ‘65.zip’

65.zip              100%[===================>]   2.37G  24.9MB/s    in 1m 40s  

2020-12-20 21:43:07 (24.3 MB/s) - ‘65.zip’ saved [2540031150/2540031150]

Archive:  65.zip
  inflating: LIST                    
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [9]:
!pwd

/content


In [10]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

word2vec_model = KeyedVectors.load_word2vec_format(datapath("/content/russian_conll17/model.bin"), binary=True)

In [11]:
from sklearn.manifold import TSNE
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook


def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({'x' : x, 'y' : y, 'color': color, **kwargs})

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig



words = sorted(word2vec_model.vocab.keys(), 
               key=lambda word: word2vec_model.vocab[word].count,
               reverse=True)[:1000]
word_vectors = word2vec_model.vectors[[word2vec_model.vocab[word].index for word in words]]

tsne_model = TSNE(n_components=2)
tsne_vectors = tsne_model.fit_transform(word_vectors)
tsne_vectors = (tsne_vectors - tsne_vectors.mean(0)) / tsne_vectors.std(0)

draw_vectors(tsne_vectors[:, 0], tsne_vectors[:, 1], token=words)

Figure(id='1002', ...)

In [12]:
import razdel
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

class Word2VecDataset(Dataset):
    def __init__(self, tokenized_texts, labels, model, max_seq_len=50):
        self.model = model
        self.vector_dim = self.model.vector_size
        self.samples = []
        assert len(tokenized_texts) != 0
        assert len(tokenized_texts) == len(labels)
        for i, (text, label) in enumerate(zip(tokenized_texts, labels)):
            text = text[:max_seq_len]
            embeddings = np.zeros((max_seq_len, self.vector_dim))
            embeddings[:len(text), :] = self.words_to_embeddings(text)
            self.samples.append((torch.FloatTensor(embeddings), torch.FloatTensor([label])))
            if i % 10000 == 0:
                print(i)

    def words_to_embeddings(self, words):
        embeddings = np.zeros((len(words), self.vector_dim))
        for i, w in enumerate(words):
            if w in self.model.vocab:
                embeddings[i] = self.model.get_vector(w)
                embeddings[i] /= np.linalg.norm(embeddings[i])
        return embeddings
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]


BATCH_SIZE = 128

train_data = Word2VecDataset(train_texts, train_labels, word2vec_model)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = Word2VecDataset(val_texts, val_labels, word2vec_model)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = Word2VecDataset(test_texts, test_labels, word2vec_model)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
0
10000
20000
30000
0
10000
20000
30000
40000


In [13]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping  


class Word2VecLSTMModel(LightningModule):
    def __init__(self, embedding_dim, hidden_dim=64):
        super().__init__()

        self.lstm_layer = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout_layer = nn.Dropout(0.2)        
        self.out_layer = nn.Linear(hidden_dim * 2, 1)
        self.loss = nn.BCEWithLogitsLoss()
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
    
    def forward(self, embeddings, labels):
        batch_size = embeddings.size(0)
        output, (final_hidden_state, final_cell_state) = self.lstm_layer(embeddings)
        final_hidden_state = final_hidden_state.transpose(0, 1)
        final_hidden_state = final_hidden_state.reshape(batch_size, -1)
        text_hidden = self.dropout_layer(final_hidden_state)
        logits = self.out_layer.forward(text_hidden)
        loss = self.loss(logits, labels)
        return loss, logits
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.valid_accuracy.update(logits, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.test_accuracy.update(logits, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)


lstm_model = Word2VecLSTMModel(word2vec_model.vector_size)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(lstm_model, train_loader, val_loader)
trainer.test(lstm_model, test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params
-----------------------------------------------------
0 | lstm_layer     | LSTM              | 85.0 K
1 | dropout_layer  | Dropout           | 0     
2 | out_layer      | Linear            | 129   
3 | loss           | BCEWithLogitsLoss | 0     
4 | valid_accuracy | Accuracy          | 0     
5 | test_accuracy  | Accuracy          | 0     
-----------------------------------------------------
85.1 K    Trainable params
0         Non-trainable params
85.1 K    Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7176, device='cuda:0'),
 'test_acc_epoch': tensor(0.7176, device='cuda:0'),
 'test_loss': tensor(0.4981, device='cuda:0'),
 'val_acc': tensor(0.7217, device='cuda:0'),
 'val_acc_epoch': tensor(0.7217, device='cuda:0'),
 'val_loss': tensor(0.4909, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.7176457643508911,
  'test_acc_epoch': 0.7176457643508911,
  'test_loss': 0.4980763792991638,
  'val_acc': 0.7217165231704712,
  'val_acc_epoch': 0.7217165231704712,
  'val_loss': 0.490856796503067}]